# Network of Things test

Start a Mosquitto container first. For example:
- Use `codes\_demo\1_start_broker.sh` to start a Mosquitto container on Raspberry Pi.
- Config files are in `mqtt_config\mqtt`.
- set `allow_anonymous true` in `mqtt_config\mqtt\config\mosquitto.conf` to allow anonymous client.

## Getting Started
What this notebook does:  
- Using a client on PC
- List connected nodes
- Send messages to remote nodes:
 - Return results (read GPIOs)via RPC mechanism.
 - Write data to remote nodes (write GPIOs).
 - Execute arbitrary code on remote nodes.

In [16]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'codes', 'micropython')))
 
import client
from collections import OrderedDict

### Start client

In [18]:
the_client = client.Client()
the_client.start()

while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')

My name is Client_366

Sending 277 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2017-02-07 15:02:17.079900'), ('kwargs', {'name': 'Client_366'}), ('message_id', '2017-02-07 15:02:17.079900'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


[Connected: ('192.168.0.114', 1883)]
[Listen to messages]
Node not ready yet.


### Prepare messages

In [19]:
# messages _____________________________________________
messages = OrderedDict()

messages['read_GPIOs'] = {'message_type': 'command',
                          'command': 'read GPIOs',
                          'kwargs': {'pins': [5, 12, 13, 14, 15, 16]},
                          'need_result': True}

messages['blink_led'] = {'message_type': 'command',
                         'command': 'blink led',
                         'kwargs': {'times': 3, 'forever': False, 'on_seconds': 0.1, 'off_seconds': 0.1}}

In [20]:
# messages['write_GPIOs'] = {'message_type': 'command',
                           # 'command': 'write GPIOs',
                           # 'kwargs': {'pins_and_values': [(2, 0), (2, 1), (2, 0),]}} 

# messages['test eval'] = {'message_type': 'eval',
                         # 'to_evaluate': '2+3',
                         # 'need_result': True}                                   

# messages['test exec'] = {'message_type': 'exec',
                         # 'to_exec': 'print("Testing exec !")'}

# with open('script_to_deploy.py') as f:
    # script = f.read()        
# messages['test upload script'] = {'message_type': 'script', 
                                  # 'script': script} 

### List connected nodes

In [21]:
the_client.node.worker.roll_call()
time.sleep(2)
remote_nodes = sorted(the_client.node.worker.contacts.keys())

print('\n[____________ Connected nodes ____________]\n')
print('\nConnected nodes:\n{}\n'.format(remote_nodes))


Sending 249 bytes
Message:
OrderedDict([('correlation_id', '2017-02-07 15:02:29.200900'), ('function', 'check_in'), ('kwargs', {'caller': 'Client_366'}), ('message_id', '2017-02-07 15:02:29.200900'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 249 bytes
Message:
OrderedDict([('correlation_id', '2017-02-07 15:02:29.200900'), ('function', 'check_in'), ('kwargs', {'caller': 'Client_366'}), ('message_id', '2017-02-07 15:02:29.200900'), ('message_type', 'function'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 172 bytes

Data received: 190 bytes
Message:
OrderedDict([('function', 'register_contact'), ('kwargs', {'name': 'NodeMCU_f1d30800', 'contact_id': 'NodeMCU_f1d30800'}), ('message_type', 'function'), ('receiver', 'Client_366'), ('sender', 'NodeMCU_f1d30800')])


Data received: 178 bytes
Message:
OrderedDict([('function', 'register_contact'), ('kwargs', {'name': 'NodeM

### Read one GPIO pin

In [22]:
for remote_node in remote_nodes:
    _, result = the_client.request(remote_node, messages['read_GPIOs']) 
    print('\nGPIO status for {}: {}\n'.format(remote_node, result.get()))


Sending 286 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-02-07 15:02:38.741900'), ('kwargs', {'pins': [5, 12, 13, 14, 15, 16]}), ('message_id', '2017-02-07 15:02:38.741900'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 286 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-02-07 15:02:38.741900'), ('kwargs', {'pins': [5, 12, 13, 14, 15, 16]}), ('message_id', '2017-02-07 15:02:38.741900'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Processed result:
OrderedDict([('correlation_id', '2017-02-07 15:02:38.741900'), ('message_id', '2017-02-07 15:02:39.678900'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('result', 'Not applicable.'), ('sender', 'Client_366')])


Sending 223 bytes
Message:


### Blink each node
[YouTube video clip](https://youtu.be/I_s-no_0L24)

In [14]:
for remote_node in remote_nodes:
    the_client.request(remote_node, messages['blink_led']) 


Sending 300 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-02-07 14:56:44.975900'), ('kwargs', {'on_seconds': 0.1, 'off_seconds': 0.1, 'forever': False, 'times': 3}), ('message_id', '2017-02-07 14:56:44.975900'), ('message_type', 'command'), ('receiver', 'Client_366'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 304 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-02-07 14:56:45.133900'), ('kwargs', {'on_seconds': 0.1, 'off_seconds': 0.1, 'forever': False, 'times': 3}), ('message_id', '2017-02-07 14:56:45.133900'), ('message_type', 'command'), ('receiver', 'NodeMCU_edca00'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 300 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-02-07 14:56:44.975900'), ('kwargs', {'on_seconds': 0.1, 'off_seconds': 0.1, 'forever': False, 'times': 3}), ('message_id', '2017-02-07 14:56:44.975900'), ('message_type', 'command

### Send out messages and get asynchonous results

In [23]:
print('\n[______________ Sending messages ______________]\n')

results = []

# send out the messages
for message in messages.values():
    for remote_node in remote_nodes:
        if remote_node != the_client.node.worker.name:
            time.sleep(0.1)  # PyCharm needs this delay.
            formatted_message, asynch_result = the_client.request(remote_node, message)
            results.append((formatted_message, asynch_result))


[______________ Sending messages ______________]


Sending 291 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-02-07 15:03:08.566900'), ('kwargs', {'pins': [5, 12, 13, 14, 15, 16]}), ('message_id', '2017-02-07 15:03:08.566900'), ('message_type', 'command'), ('need_result', True), ('receiver', 'NodeMCU_1dsc000'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 288 bytes
Message:
OrderedDict([('command', 'read GPIOs'), ('correlation_id', '2017-02-07 15:03:08.890900'), ('kwargs', {'pins': [5, 12, 13, 14, 15, 16]}), ('message_id', '2017-02-07 15:03:08.890900'), ('message_type', 'command'), ('need_result', True), ('receiver', 'NodeMCU_8a00'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 249 bytes
Message:
OrderedDict([('correlation_id', '2017-02-07 15:03:08.566900'), ('message_id', '126958'), ('message_type', 'result'), ('receiver', 'Client_366'), ('reply_to', 'NodeMCU_1dsc000'), ('result', [[5, 0], [12, 1], [13, 1

### Actually get the results

In [24]:
# collect and print results        
print('\n[_________ Wait few seconds for reply _________]\n')
for (message, result) in results:
    try:
        if message.get('need_result'):
            print('\n[Result for request]:\n___Request___:\n{0}\n___Result____:\n{1}\n'.format(message,
                                                                                               result.get() if result else None))
    except Exception as e:
        print('\n[{}]\nMessage:\n{}'.format(e, message))
        
# Wait a while
time.sleep(3)        


[_________ Wait few seconds for reply _________]


[Result for request]:
___Request___:
{'message_id': '2017-02-07 15:03:08.566900', 'need_result': True, 'command': 'read GPIOs', 'kwargs': {'pins': [5, 12, 13, 14, 15, 16]}, 'receiver': 'NodeMCU_1dsc000', 'message_type': 'command', 'reply_to': 'Client_366', 'correlation_id': '2017-02-07 15:03:08.566900', 'sender': 'Client_366'}
___Result____:
[[5, 0], [12, 1], [13, 1], [14, 1], [15, 0], [16, 0]]


[Result for request]:
___Request___:
{'message_id': '2017-02-07 15:03:08.890900', 'need_result': True, 'command': 'read GPIOs', 'kwargs': {'pins': [5, 12, 13, 14, 15, 16]}, 'receiver': 'NodeMCU_8a00', 'message_type': 'command', 'reply_to': 'Client_366', 'correlation_id': '2017-02-07 15:03:08.890900', 'sender': 'Client_366'}
___Result____:
[[5, 1], [12, 1], [13, 1], [14, 1], [15, 0], [16, 1]]


[Result for request]:
___Request___:
{'message_id': '2017-02-07 15:03:09.268900', 'need_result': True, 'command': 'read GPIOs', 'kwargs': {'pins': [5, 

### Stop the demo

In [25]:
# Stopping
the_client.stop()
the_client = None
print('\n[________________ Demo stopped ________________]\n')

[Closed: ('192.168.0.114', 1883)]
[________________ Demo stopped ________________]


